In [1]:
eventNoList = [
          "001", # Lilac Wildfire 2017
          "002", # Cranston Wildfire 2018
          "003", # Holy Wildfire 2018
          "004", # Hurricane Florence 2018
          "005", # 2018 Maryland Flood
          "006", # Saddleridge Wildfire 2019
          "007", # Hurricane Laura 2020
          "008" # Hurricane Sally 2020
]


In [2]:
import ir_datasets
import pandas as pd
import lucene
from org.apache.lucene.analysis.standard import StandardAnalyzer
from org.apache.lucene.analysis.en import EnglishAnalyzer
from org.apache.lucene.index import IndexWriter , IndexWriterConfig
from org.apache.lucene.store import SimpleFSDirectory , FSDirectory
import org.apache.lucene.document as document 
from org.apache.lucene.document import Document, Field
from java.io import File
from nltk.stem import PorterStemmer
ps = PorterStemmer()

In [3]:
import requests

# Gets the list of days for a specified event number, e.g. '001'
def getDaysForEventNo(eventNo):

  # We will download a file containing the day list for an event
  url = "http://trecis.org/CrisisFACTs/CrisisFACTS-"+eventNo+".requests.json"

  # Download the list and parse as JSON
  dayList = requests.get(url).json()

  # Print each day
  # Note each day object contains the following fields
  #   {
  #      "eventID" : "CrisisFACTS-001",
  #      "requestID" : "CrisisFACTS-001-r3",
  #      "dateString" : "2017-12-07",
  #      "startUnixTimestamp" : 1512604800,
  #      "endUnixTimestamp" : 1512691199
  #   }

  return dayList

In [4]:
lucene.initVM()

In [23]:
def text_end(source):
    stop = ( '.','?','!')
    flag=0
    i = 0
    M = max(len(s) for s in stop)
    L = len(source)
    text=" "
    while i <= L:
        m = M
        while m > 0:
            chunk = source[i:i + m]
            if (chunk in stop) and (i==(L-1)):
                flag=1
                break
            m -= 1
        else:
            m = 1
        i += m
    if(flag==0):
        text=source+"."
    else:
        text=source
    #print(text)
    return text

In [24]:
def text_preproc(query): ## FOR CONTENT
    query=re.sub(r'http\S+', '',query)
    query=query.encode("ascii","ignore")
    query=query.decode()
    query=query.casefold()
    tokens=word_tokenize(query)
    #print(tokens)
    filtered_text=""
    for token in tokens:
        #print("Token",token)
        if (('http' not in token) and ('@' not in token) and ('*'not in token)):
            token=ps.stem(token)
            token=token.strip("/")
            token=token.replace(".","")
            token=token.replace("?","")
            token=token.replace("+","")
            token=token.replace("!","")
            token=token.replace("(","")
            token=token.replace(")","")
            filtered_text=filtered_text+" "+token
    return filtered_text

In [26]:
def text_clean(query): ## FOR FACTTEXT
    query=query.encode("ascii","ignore")
    query=query.decode()
    tokens=word_tokenize(query)
    #print(tokens)
    filtered_text=""
    for token in tokens:
        #print("Token",token)
        if (('@' not in token) and ('*'not in token)):
            token=token.strip("/")
            #token=token.replace("'","")
            token=token.replace("+","")
            token=token.replace("(","")
            token=token.replace(")","")
            filtered_text=filtered_text+" "+token
    filtered_text=text_end(filtered_text)
    #print(filtered_text)
    return filtered_text[0:200]

In [18]:
def closeWriter():
    writer.close()

In [10]:
def index_void_text(docid,cleanDoc,timestamp,requestId):
    doc=document.Document()
    #print(docid)
    #print(cleanDoc)
    doc.add(document.Field("Docid",docid,document.TextField.TYPE_STORED))
    doc.add(document.Field("Content",cleanDoc,document.TextField.TYPE_STORED))
    doc.add(document.Field("Timestamp",str(timestamp),document.TextField.TYPE_STORED))
    doc.add(document.Field("RequestId",requestId,document.TextField.TYPE_STORED))
    writer.addDocument(doc)

In [11]:
def index(docid,cleanDoc,timestamp,requestId,facttext):
    doc=document.Document()
    #print(docid)
    #print(cleanDoc)
    doc.add(document.Field("Docid",docid,document.TextField.TYPE_STORED))
    doc.add(document.Field("Content",cleanDoc,document.TextField.TYPE_STORED))
    doc.add(document.Field("Timestamp",str(timestamp),document.TextField.TYPE_STORED))
    doc.add(document.Field("RequestId",requestId,document.TextField.TYPE_STORED))
    doc.add(document.Field("Facttext",facttext,document.TextField.TYPE_STORED))
    writer.addDocument(doc)

In [12]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
import re
   
# Input text - to summarize 
def summary(text,itemsAsDataFrame,request):
#     stopWords = set(stopwords.words("english"))
#     words = word_tokenize(text)
#     i=0
   
# # Creating a frequency table to keep the 
# # score of each word
   
#     freqTable = dict()
#     for word in words:
#         word = word.lower()
#         if word in stopWords:
#             continue
#         if word in freqTable:
#             freqTable[word] += 1
#         else:
#             freqTable[word] = 1
   
# Creating a dictionary to keep the score
# of each sentence
    sentences =text.split(". ")
    sentenceValue = dict()
    sentenceId=dict()
    sentenceTimestamp=dict()
    sentenceOriginal=dict()
    #print(sentences)
    print(len(sentences))
    print((len(itemsAsDataFrame)))
    i=0
    for sentence in sentences:
        sentenceId[sentence]=itemsAsDataFrame['doc_id'][i]
        i+=1
    i=0
    for sentence in sentences:
        sentenceTimestamp[sentence]=itemsAsDataFrame['unix_timestamp'][i]
        i+=1
    i=0
    for sentence in sentences:
#         print("stemed")
#         print(sentence)
#         print("original")
#         print((itemsAsDataFrame['text'][i]))
        sentenceOriginal[sentence]=text_clean(itemsAsDataFrame['text'][i])
        i+=1
    
            
    #print(sentenceId)   
#     for sentence in sentences:
#         for word, freq in freqTable.items():
#             if word in sentence.lower():
#                 if sentence in sentenceValue:
#                     sentenceValue[sentence] += freq
#                 else:
#                     sentenceValue[sentence] = freq
        
   
   
#     #print(len(sentenceValue))
#     sumValues = 0
#     for sentence in sentenceValue:
#         sumValues += sentenceValue[sentence]
    
   
# # Average value of a sentence from the original text
    
#     average = int(sumValues / len(sentenceValue))
# # Storing sentences into our summary.
#     summary =[]
#     doc_id=[]
#     timestamp=[]
    for sentence in sentences:
        docid=sentenceId[sentence]
        content=sentence
        timestamp=sentenceTimestamp[sentence]
        requestId=request["requestID"]
        facttext=sentenceOriginal[sentence]
        index(docid,content,timestamp,requestId,facttext)
#             doc_id.append(sentenceId[sentence])
#             summary.append(sentence)
#             timestamp.append(sentenceTimestamp[sentence])
        
            
    return summary,docid,timestamp

In [13]:
def sentence_end(source):
    stop = ( '.')
    flag=0
    i = 0
    M = max(len(s) for s in stop)
    L = len(source)
    text=" "
    while i <= L:
        m = M
        while m > 0:
            chunk = source[i:i + m]
            if (chunk in stop) and (i==(L-1)):
                flag=1
                break
            m -= 1
        else:
            m = 1
        i += m
    if(flag==0):
        text=source+"."
    else:
        text=source
    #print(text)
    return text

In [27]:
def fact_text(itemsAsDataFrame,dataset,request,path):
    text=" "
    for j in range(0,len(itemsAsDataFrame)):
        s=text_preproc(itemsAsDataFrame['text'][j])
        text=text+" " +sentence_end(s)
    #print(len(text))
#     if len(text)==41:
#         for i in range(0,len(itemsAsDataFrame)):
#             docid=itemsAsDataFrame['doc_id'][i]
#             content=itemsAsDataFrame['text'][i]
#             timestamp=itemsAsDataFrame['unix_timestamp'][i]
#             requestId=request["requestID"]
#             index_void_text(docid,content,timestamp,requestId)
#         closeWriter()
        
    else:
        
        k,l,time=summary(text,itemsAsDataFrame,request)
        closeWriter()
    #query_retrieval(k,l,dataset,request,time)

In [15]:
def data(event,request,path):
    dataset = ir_datasets.load(event)
    itemsAsDataFrame = pd.DataFrame(dataset.docs_iter())
    fact_text(itemsAsDataFrame,dataset,request,path)
    

In [28]:
for eventNo in eventNoList: # for each event
    dayList = getDaysForEventNo(eventNo) # get the list of days
    print("Event "+eventNo)
    #print(dayList[0]['requestID'])
    for day in dayList: # for each day
        event="crisisfacts/"+eventNo+"/"+day["dateString"]
        path="crisisfacts_"+str(eventNo)+"_"+str(day["dateString"])+"/"
        indexPath=File(path).toPath()
        indexDir=FSDirectory.open(indexPath)
        writerConfig=IndexWriterConfig(EnglishAnalyzer())
        writer=IndexWriter(indexDir,writerConfig)
        data(event,day,path)
        print("##")
        
    
        
    

        
    

Event 001
7288
7288
##
19231
19231
##
5839
5839
##
4407
4407
##
3394
3394
##
2805
2805
##
2658
2658
##
2728
2728
##
2665
2665
##
Event 002
5056
5056
##
7866
7866
##
7433
7433
##
5238
5238
##
4691
4691
##
251
251
##
Event 003
4402
4402
##
5727
5727
##
5925
5925
##
5984
5984
##
6233
6233
##
4023
4023
##
195
195
##
Event 004
7959
7959
##
503
503
##
1559
1559
##
3329
3329
##
6056
6056
##
8668
8668
##
38738
38738
##
55465
55465
##
17000
17000
##
39000
39000
##
58610
58610
##
20301
20301
##
13528
13528
##
18202
18202
##
710
710
##
Event 005
7380
7380
##
16407
16407
##
10226
10226
##
7757
7757
##
Event 006
6993
6993
##
15014
15014
##
9364
9364
##
6998
6998
##
Event 007
46021
46021
##
16161
16161
##
Event 008
2215
2215
##
8656
8656
##
7678
7678
##
11000
11000
##
22399
22399
##
10000
10000
##
17087
17087
##
5437
5437
##
